In [1]:
import MeCab
import pandas as pd
import gensim
import gensim.corpora as corpora
from pprint import pprint
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

m = MeCab.Tagger()

target_tags = [
    'NNG',  # 일반 명사
    'NNP',  # 고유 명사
    'NNB',  # 의존 명사
    'NR',  # 수사
    'NP',  # 대명사
    'VV',  # 동사
    'VA',  # 형용사
    'MAG',  # 일반 부사
    'MAJ',  # 접속 부사
]

def parse_sentence(sentence, target_tags, stop_word):
    result = m.parse(sentence)
    temp = result.split('\n')
    temp_2 = [ sentence.split('\t') for sentence in temp]
    words = [ sentence[0] for sentence in temp_2 ]
    morphs = [ sentence[1].split(',')[0]
               for sentence in temp_2
               if len(sentence) > 1]
    morphs = [ morph for morph in morphs if morph in target_tags ]
    words = words[:len(morphs)]



    word_morph = [ (word,morph)
                   for morph, word in zip(morphs, words)
                   if word not in stop_word ]
    return word_morph

def extract_word_list(lyrics, target_tags, stop_word):
    result = []
    try:
        for idx in range(len(lyrics)):
            word_morph_list = parse_sentence(lyrics[idx], target_tags, stop_word)
            word = [ word_morph[0] for word_morph in word_morph_list if len(word_morph[0]) > 1]
            result.append(word)
    except:
        print(idx, '해당 인덱스에서 오류가 났습니다.')
    return result

df = pd.read_csv('../data/발라드.csv')
print(df.head())
lyrics = df['lyrics'].values
stop_word = ['것', '을', '겠', '은', '.', '는', ',']
word = extract_word_list(lyrics, target_tags, stop_word)

def make_bigram(word):
    return gensim.models.Phrases(word, min_count=5, threshold=100)

def make_trigram(word):
    bigram = gensim.models.Phrases(word, min_count=5, threshold=100)
    return gensim.models.Phrases(bigram[word], threshold=100)
#print(bigram)
def make_trigram_list(word, bigram_mod, trigram_mod):
    trigram_list = []
    for idx in range(len(word)):
        trigram_list.append(trigram_mod[bigram_mod[word[idx]]])
    return trigram_list

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = make_bigram(word)
trigram_mod = make_trigram(word)
trigram_list = make_trigram_list(word, bigram_mod, trigram_mod)
print(trigram_list[0])

# Create Dictionary and corpus for topic modeling
id2word = corpora.Dictionary(trigram_list)
print(id2word[0])

corpus = [id2word.doc2bow(text) for text in trigram_list]
#print(corpus)

# View
print('corpus', corpus[:1])
print('id2word', id2word[corpus[0][0][0]])

#
temp = [[(id, id2word[id], freq) for id, freq in cp][:10] for cp in corpus[:1]]
print('상위 10개 단어', temp)

# gensim.models.ldamodel
#class gensim.models.ldamodel.LdaModel(corpus=None,
#                                     num_topics=100,
#                                     id2word=None,
#                                     distributed=False,
#                                     chunksize=2000,
#                                     passes=1,
#                                     update_every=1,
#                                     alpha='symmetric',
#                                     eta=None,
#                                     decay=0.5,
#                                     offset=1.0,
#                                     eval_every=10,
#                                     iterations=50,
#                                     gamma_threshold=0.001,
#                                     minimum_probability=0.01,
#                                     random_state=None,
#                                     ns_conf=None,
#                                     minimum_phi_value=0.01,
#                                     per_word_topics=False,
#                                    callbacks=None,
#                                     dtype=<class 'numpy.float32'>)

print(id2word)
NUM_TOPIC = 3
lda_model = gensim.models.ldamodel.LdaModel(iterations=200,
                                            corpus=corpus,
                                            id2word=id2word,
                                            num_topics=NUM_TOPIC,  #만약에 토픽이 8개라면, 그러면 그 토픽은 무엇이니?
                                            random_state=100,
                                            chunksize=400,
                                            passes=100,  # 중복된 토픽이 나오는 경우, 에폭을 늘려야한다.
                                            alpha='auto',
                                            per_word_topics=True
                                            )

# Print the Keyword in the 10 topics
# ldamodel이 정한 토픽중 앞쪽 순서 10개의 토픽에 해당되는 키워드들입니다.
# 각 키워드들에는 가중치가 정해져있습니다.
# 이 가중치들을 바탕으로 문서의 토픽을 분류합니다.
pprint(lda_model.print_topics())

doc_lda = lda_model[corpus]
print(doc_lda)

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)


vis

   Unnamed: 0                                             lyrics  이별유무  긍부정
0           0  숨을 쉴 수 없죠. 그대 생각나면. 잠을 잘 수 없고. 도무지 살 수가 없죠. 살아...     1    0
1           1  여기서 나갈 수가 없다.아무리 노력한다.나는 거의 끝나겠어.그래서 거짓말에 대한 요...     0    2
2           2  밤이 뱉는. 조용한 속삭임이.왠지 슬퍼. 네가 더그리운 밤. 괜찮을줄 알았었는데.여...     1    0
3           3  나 기다리지 마요 우리의 추억에 날. 나 울리지는 마요. 그 기억 너머의 우리. 좋...     1    0
4           4  나 기다리지 마요 우리의 추억에 날. 나 울리지는 마요. 그 기억 너머의 우리. 좋...     1    0
['그대', '생각나', '도무지', '살아오', '동안', '그대', '는데', '전부', '는데', '그댈', '만난', '잘못', '헤어질', '만나', '지나', '함께', '지낸', '숱한', '그게', '생각나', '가끔']
가끔
corpus [[(0, 1), (1, 1), (2, 2), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1)]]
id2word 가끔
상위 10개 단어 [[(0, '가끔', 1), (1, '그게', 1), (2, '그대', 2), (3, '그댈', 1), (4, '는데', 2), (5, '도무지', 1), (6, '동안', 1), (7, '만나', 1), (8, '만난', 1), (9, '살아오', 1)]]
Dictionary(3657 unique tokens: ['가끔', '그게', '그대', '그댈', '는데']...)
[(0,
  '0.019*"

C:\nlp2\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.036816 -0.097244       1        1  42.568197
1     -0.092937  0.072737       2        1  41.134427
2      0.129753  0.024507       3        1  16.297376, topic_info=    Term        Freq       Total Category  logprob  loglift
216   사람  205.000000  205.000000  Default  30.0000  30.0000
88    처럼  230.000000  230.000000  Default  29.0000  29.0000
81    이별   60.000000   60.000000  Default  28.0000  28.0000
329   햇살   41.000000   41.000000  Default  27.0000  27.0000
2     그대  514.000000  514.000000  Default  26.0000  26.0000
..   ...         ...         ...      ...      ...      ...
178   모르   15.714373   81.757652   Topic3  -5.7506   0.1650
180   아직   16.090872  115.324312   Topic3  -5.7269  -0.1553
127  이렇게   16.030483  111.548732   Topic3  -5.7306  -0.1258
108   우리   17.066218  397.965385   Topic3  -5.6680  -1.3351
98    다시   16.291341  208.251731   Topic3  -5.7145  -0.7339

[256 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
223       3  0.847650        가까이
148       1  0.797211         가득
148       3  0.190379         가득
2118      3  0.947392        건가요
355       1  0.909072        걸어가
...     ...       ...        ...
1123      2  0.873153         힘든
3557      2  0.938194  힘든_새로움_버렸
267       1  0.112130         힘들
267       2  0.809825         힘들
267       3  0.087212         힘들

[352 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])

In [11]:
pprint(lda_model.print_topics())

doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.019*"우리" + 0.017*"사랑" + 0.016*"그대" + 0.011*"모든" + 0.010*"당신" + 0.010*"시간" '
  '+ 0.009*"기억" + 0.009*"처럼" + 0.009*"다시" + 0.008*"오늘"'),
 (1,
  '0.028*"사랑" + 0.024*"그대" + 0.012*"마음" + 0.011*"우리" + 0.010*"너무" + 0.009*"사람" '
  '+ 0.008*"에게" + 0.007*"는데" + 0.007*"함께" + 0.006*"시간"'),
 (2,
  '0.011*"처럼" + 0.011*"사람" + 0.011*"마음" + 0.009*"사랑" + 0.008*"오늘" + 0.007*"하루" '
  '+ 0.007*"생각" + 0.005*"함께" + 0.005*"햇살" + 0.005*"매일"')]


[(0,
  '0.019*"우리" + 0.017*"사랑" + 0.016*"그대" + 0.011*"모든" + 0.010*"당신" + 0.010*"시간" + 0.009*"기억" + 0.009*"처럼" + 0.009*"다시" + 0.008*"오늘"'),
 (1,
  '0.028*"사랑" + 0.024*"그대" + 0.012*"마음" + 0.011*"우리" + 0.010*"너무" + 0.009*"사람" + 0.008*"에게" + 0.007*"는데" + 0.007*"함께" + 0.006*"시간"'),
 (2,
  '0.011*"처럼" + 0.011*"사람" + 0.011*"마음" + 0.009*"사랑" + 0.008*"오늘" + 0.007*"하루" + 0.007*"생각" + 0.005*"함께" + 0.005*"햇살" + 0.005*"매일"')]